In [3]:
from sequence_extractor import *
import pickle
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import keras
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from keras import regularizers
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Conv1D, Conv2D, Flatten, MaxPooling1D, MaxPooling2D, Dropout, Activation
import gzip

Using TensorFlow backend.


In [4]:
# Deep Learning model using Hai Wang's benchmark

def hw_model(in_shape):
    model=Sequential()
    model.add(Conv2D(64,kernel_size=(8,5),padding='valid',input_shape=in_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(64,kernel_size=(8,1),padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(8,1),strides=(8,1),padding='same'))
    model.add(Dropout(0.25))

    model.add(Conv2D(128,kernel_size=(8,1),padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(128,kernel_size=(8,1),padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(8,1),strides=(8,1),padding='same'))
    model.add(Dropout(0.25))

    model.add(Conv2D(64,kernel_size=(8,1),padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64,kernel_size=(8,1),padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(8,1),strides=(8,1),padding='same'))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dense(1))#2))
    model.add(Activation('sigmoid'))#'softmax'))
    adam1=keras.optimizers.Adam(lr=0.00001)
    model.compile(loss='binary_crossentropy', optimizer=adam1, metrics=['accuracy'])
    return model

In [5]:
f = gzip.open('PM-genes-first2000.pklz','rb')
loadData = pickle.load(f)
f.close()

f = gzip.open('PM-genes-first2000-label.pklz','rb')
loadLabel = pickle.load(f)
f.close()

In [13]:
X = []
Y = []

for i in sorted(loadData):
    X.append(loadData[i])
    
for i in sorted(loadLabel):
    Y.append(loadLabel[i])
    
X = np.array(X)
Y = np.array(Y)

print len(loadData), X.shape, Y.shape

12 (12,) (12,)


In [ ]:
# fit the deep learning model

batchSize=50
erl_stop = EarlyStopping(monitor='val_acc', min_delta=0, patience=30, verbose=0, mode='max')
hist = model.fit(x_train[rand], y_train[rand], epochs=1000, validation_data=[x_val[rand_val],y_val[rand_val]], batch_size=batchSize, verbose=1, shuffle=True, 
                callbacks=[erl_stop])